In [1]:
from img_data_class import ImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import torch
from torch import nn
import matplotlib.pyplot as plt
from model_class import ConditionalVAE
import torch.optim as optim
import torch.nn.functional as F
import tqdm

c:\Users\Tomislav\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EPOCHS = 1
BATCH_SIZE = 32


In [3]:
tr = transforms.Compose(
    [
     transforms.Resize((256,256)),
     transforms.ToTensor()
    ],
    )
random_erase_tr = transforms.RandomErasing(1)

In [4]:
train = ImageDataset('data/train',tr)

In [5]:
train_dl = DataLoader(train,BATCH_SIZE,shuffle=True)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
model = ConditionalVAE(3,64,device).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [9]:
def loss_function(recon_x, x, mu, log_var):
    bce = nn.CrossEntropyLoss(reduction='sum')
    BCE = bce(recon_x.flatten(1), x.flatten(1))
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD
def augment_image(random_erase_fn,img):
    img_list = [random_erase_fn(i) for i in img]
    return torch.stack(img_list)

In [10]:
for epoch in range(EPOCHS):
    train_epoch_loss = 0
    for batch_idx,img in tqdm.tqdm(enumerate(train_dl),total=len(train_dl)):
        model.train()
        #plt.imshow(make_grid(img).permute(1,2,0))
        input_img = augment_image(random_erase_tr,img).to(device)
        img = img.to(device)
        
        optimizer.zero_grad()
        recon_img, mu , log_var = model(input_img)
        loss = loss_function(recon_img, img, mu, log_var)
        train_epoch_loss += loss.item()
        loss.backward()
        optimizer.zero_grad()
        

  0%|          | 1/2533 [00:08<6:11:19,  8.80s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 4.00 GiB total capacity; 2.93 GiB already allocated; 0 bytes free; 3.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import os

In [ ]:
len(os.listdir('data/train/'))

162079

In [ ]:
len(os.listdir('data/test/'))

40520